In [1]:
from keras import Input, layers
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from pickle import dump, load
import os
import numpy as np
from keras.layers.merge import add
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from numpy import array

Using TensorFlow backend.


In [2]:
max_length=34
vocab_size=7579

## Load wordtoix and ixtoword

In [3]:
with open("./wordtoix.pkl", "rb") as encoded_pickle:
    wordtoix = load(encoded_pickle)

In [4]:
with open("./ixtoword.pkl", "rb") as encoded_pickle:
    ixtoword = load(encoded_pickle)

## Def some functions

In [5]:
def save_model(model, filename):
    model_json = model.to_json()
    with open(filename + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(filename + ".h5")
    print("Saved model to disk")

In [6]:
def load_descriptions(filename):
    file = open(filename, 'r')
    doc = file.read()
    file.close()
    
    mapping = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        if len(line) < 2:
            continue
        image_id, image_desc = tokens[0], tokens[1:]
        image_id = image_id.split('.')[0]
        image_desc = ' '.join(image_desc)
        if image_id not in mapping:
            mapping[image_id] = list()
        mapping[image_id].append(image_desc)
    return mapping

In [7]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

## Load glove vectors

In [8]:
glove_dir = './'
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [9]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

# Define model

In [10]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 200)      1515800     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
__________________________________________________________________________________________________
dropout_2 

In [12]:
model.layers[2]

In [13]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [15]:
train_descriptions = load_descriptions('train_captions.txt')
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=6000


In [16]:
train_features = load(open("./encoded_train_images.pkl", "rb"))

## Train model

In [17]:
epochs = 10
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [18]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    save_model(model, './model_weights/model_' + str(i))

Epoch 1/1
2000/2000 [==============================] - 459s 230ms/step - loss: 4.5894
Saved model to disk
Epoch 1/1
2000/2000 [==============================] - 2073s 1s/step - loss: 3.7540
Saved model to disk
Epoch 1/1
2000/2000 [==============================] - 538s 269ms/step - loss: 3.4604
Saved model to disk
Epoch 1/1
2000/2000 [==============================] - 527s 263ms/step - loss: 3.2694
Saved model to disk
Epoch 1/1
2000/2000 [==============================] - 483s 242ms/step - loss: 3.1332
Saved model to disk
Epoch 1/1
2000/2000 [==============================] - 470s 235ms/step - loss: 3.0247
Saved model to disk
Epoch 1/1
2000/2000 [==============================] - 544s 272ms/step - loss: 2.9393
Saved model to disk
Epoch 1/1
2000/2000 [==============================] - 531s 265ms/step - loss: 2.8687
Saved model to disk
Epoch 1/1
2000/2000 [==============================] - 554s 277ms/step - loss: 2.8176
Saved model to disk
Epoch 1/1
2000/2000 [===========================